In [1]:
%run ../Python_files/util_data_storage_and_load.py

In [2]:
%run ../Python_files/load_dicts.py

In [3]:
%run ../Python_files/util.py

In [4]:
# load logit_route_choice_probability_matrix
P = zload('../temp_files/logit_route_choice_probability_matrix.pkz')

In [5]:
import numpy as np
from numpy.linalg import inv

P = np.matrix(P)

In [6]:
np.size(P, 0), np.size(P, 1)

(56, 140)

In [7]:
import numpy as np
from numpy.linalg import inv

def samp_cov(x):
    """
    x: sample matrix, each column is a link flow vector sample; 24 * K
    K: number of samples
    S: sample covariance matrix
    ----------------
    return: inv(S)
    ----------------
    """
    x = np.matrix(x)
    K = np.size(x, 1)
    x_mean = sum(x[:,k] for k in range(K)) / K
    S = sum(np.dot(x[:,k] - x_mean, np.transpose(x[:,k] - x_mean)) for k in range(K)) / (K - 1)
    return S

In [8]:
# load path-link incidence matrix
A = zload('../temp_files/path-link_incidence_matrix.pkz')

In [9]:
# load link counts data

import json

with open('../temp_files/link_day_minute_Oct_dict_JSON.json', 'r') as json_file:
    link_day_minute_Oct_dict_JSON = json.load(json_file)

In [10]:
week_day_Oct_list = [1, 2, 3, 4, 5, 8, 9, 10, 11, 12, 15, 16, 17, 18, 19, 22, 23, 24, 25, 26, 29, 30, 31]

In [11]:
link_day_minute_Oct_list = []
for link_idx in range(24):
    for day in week_day_Oct_list: 
        for minute_idx in range(120):
            key = 'link_' + str(link_idx) + '_' + str(day)
            link_day_minute_Oct_list.append(link_day_minute_Oct_dict_JSON[key] ['PM_flow_minute'][minute_idx])

In [12]:
len(link_day_minute_Oct_list)

66240

In [13]:
x = np.matrix(link_day_minute_Oct_list)
x = np.matrix.reshape(x, 24, 2760)

x = np.nan_to_num(x)
y = np.array(np.transpose(x))
y = y[np.all(y != 0, axis=1)]
x = np.transpose(y)
x = np.matrix(x)

In [14]:
np.size(x, 0), np.size(x, 1)

(24, 2760)

In [15]:
x[:,:2]

matrix([[  1906.55552909,   2047.264259  ],
        [  2330.54847006,   2029.3018055 ],
        [  3082.55062179,   4583.88210519],
        [  3917.44580334,   4366.36268353],
        [  2638.24125164,   1513.09744273],
        [  1383.62873995,   1082.48784785],
        [  2581.54738121,   3010.02092677],
        [  2404.67435344,   1976.28653831],
        [ 10816.61022581,   9112.47368828],
        [  9304.03942826,   4726.30755066],
        [  9736.42601349,   9120.25761148],
        [  5499.76317369,   5478.4232752 ],
        [  2262.44657876,   2164.49716615],
        [  3108.21656209,   3520.38169892],
        [  6286.63693276,   5925.14907365],
        [  5959.64780864,   5584.6852888 ],
        [  2544.16798269,   2722.30327516],
        [  1707.06994851,   2090.69525365],
        [  2984.37374715,   3305.84021667],
        [  5313.09970383,   5408.15875339],
        [  8565.95292932,   9350.36504538],
        [  9445.13758743,   9513.80564911],
        [  1294.02119972,   1943

In [16]:
np.size(A,0), np.size(A,1)

(24, 140)

In [17]:
from gurobipy import *

L = 56  # dimension of lam

K = np.size(x, 1)
S = samp_cov(x)
inv_S = inv(S)

A_t = np.transpose(A)
P_t = np.transpose(P)
# PA'
PA_t = np.dot(P, A_t)
# AP_t
AP_t = np.transpose(PA_t)

Q = np.dot(np.dot(PA_t, inv_S), AP_t)
b = sum([np.dot(np.dot(PA_t, inv_S), x[:, k]) for k in range(K)])


model = Model("OD_matrix_estimation")

lam = []
for l in range(L):
    lam.append(model.addVar(name='lam_' + str(l)))

model.update() 

# Set objective: (K/2) lam' * Q * lam - b' * lam
obj = 0
for i in range(L):
    for j in range(L):
        obj += (1.0 /2) * K * lam[i] * Q[i, j] * lam[j]
for l in range(L):
    obj += - b[l] * lam[l]
model.setObjective(obj)

# Add constraint: lam >= 0
for l in range(L):
    model.addConstr(lam[l] >= 0)

model.update() 

model.optimize()

lam_list = []
for v in model.getVars():
    print('%s %g' % (v.varName, v.x))
    lam_list.append(v.x)
print('Obj: %g' % obj.getValue())

Optimize a model with 56 rows, 56 columns and 56 nonzeros
Model has 1596 quadratic objective terms
Coefficient statistics:
  Matrix range    [1e+00, 1e+00]
  Objective range [1e-01, 3e+01]
  Bounds range    [0e+00, 0e+00]
  RHS range       [0e+00, 0e+00]
Presolve removed 56 rows and 0 columns
Presolve time: 0.00s
Presolved: 0 rows, 56 columns, 0 nonzeros
Presolved model has 1596 quadratic objective terms
Ordering time: 0.00s

Barrier statistics:
 Free vars  : 24
 AA' NZ     : 2.760e+02
 Factor NZ  : 3.000e+02
 Factor Ops : 4.900e+03 (less than 1 second per iteration)
 Threads    : 1

                  Objective                Residual
Iter       Primal          Dual         Primal    Dual     Compl     Time
   0  -2.80054176e+06  0.00000000e+00  3.91e+03 3.28e+02  9.04e+05     0s
   1   6.03769080e+05 -1.90865188e+06  5.85e+02 1.35e+02  1.80e+05     0s
   2   3.05238218e+05 -1.08060431e+06  2.12e+01 4.87e+00  2.82e+04     0s
   3  -1.62412363e+05 -3.77957444e+05  2.12e-05 5.68e-14  3.8

In [18]:
# write estimation result to file
n = 8  # number of nodes
with open('../temp_files/OD_demand_matrix_Oct_weekday_PM.txt', 'w') as the_file:
    idx = 0
    for i in range(n + 1)[1:]:
        for j in range(n + 1)[1:]:
            if i != j: 
                the_file.write("%d,%d,%f\n" %(i, j, lam_list[idx]))
                idx += 1

In [19]:
lam_list

[1.1204514851558562e-09,
 6505.625137772082,
 1.2298865489811992e-09,
 324.1191442414438,
 3510.6360998481223,
 1.2547356884385502e-05,
 3.601321640950366e-07,
 0.00010962235128691555,
 784.2984475859631,
 5505.679321892668,
 545.8613849135182,
 644.45153966064,
 1.2920712782498909e-05,
 1.32304450731999e-05,
 6495.036649728974,
 1955.31762204183,
 1309.396823322682,
 5650.345892495554,
 4540.724951322871,
 7879.250298212266,
 1.2645944345593028e-05,
 1140.1448817342916,
 5157.920474556963,
 5.305175379894592,
 3554.6291586946095,
 4.479609895427217,
 7.130830567889904e-06,
 2145.6467477089727,
 0.3962839710729362,
 0.41994374604164086,
 4482.543836841471,
 2.8924579806069772,
 7832.3778338479015,
 7.0581292303277185e-06,
 7.06016976282194e-06,
 4.2631049619749633e-07,
 4.263867247016181e-07,
 4.164874559778149e-07,
 6290.696699912707,
 5.490075970301632,
 6590.310685352915,
 1214.560968346564,
 8.89758302519255e-07,
 8.8992689440724e-07,
 14609.14372789815,
 1061.9068389078745,
 2256.